In [31]:
import server
import requests
import os
import random

STOPWORDS = server.controller.tokenizer.STOPWORDS

In [32]:
from server.controller.tokenizer import *
from server.models import db, Post, Tag, User, PostTag

In [33]:
os.environ['FLASK_CONFIG'] = './local/server-config-dev.yaml'
app = server.create_app()

In [34]:
models = server.models
db = server.models.db

In [35]:
with app.app_context():
    q = db.session.query(models.Tag)
    q = q.filter_by(has_explicit=True)
    tags = q.all()

tag_names = sorted([t.tag for t in tags])

/Users/zen/anaconda3/lib/python3.6/site-packages/pymysql/cursors.py:166: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  result = self._query(query)


In [36]:
tag_names

['apps',
 'aws',
 'containers',
 'docker',
 'healthtech',
 'hippa',
 'jill',
 'tutorial']

In [37]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

In [72]:
title = 'deploying microservice apps in GCP with docker kubernetes'

In [73]:
tokens = server.controller.tokenizer.get_tokens(title)

In [74]:
with app.app_context():
    original_words = set(t['original'].lower() for t in tokens if t['original']!=t['output'])
    q = db.session.query(Tag)
    q = q.filter(db.func.lower(Tag.tag).in_(original_words))
    tags = q.all()
    
reverse_tag_lookup = {tag.tag.lower(): tag for tag in tags}

In [75]:
add_tokens = []
for token in tokens:
    if token['original'] in reverse_tag_lookup and token['original'] != token['output']:
        tag_obj = reverse_tag_lookup[token['original']]
        add_tokens.append(
            {
                'output': tag_obj.tag,
                'tag_id': tag_obj.id
            }
        )
        
tokens.extend(add_tokens)

In [76]:
tag_names = [t['output'] for t in tokens]

In [78]:
# search_tags = k_unique(tag_names, k=2)
search_tags = tag_names

with app.app_context():
    q = db.session.query(models.Post, db.func.count(models.Tag.tag))
    q = q.select_from(models.Post)
    q = q.join(models.Post.post_tags)
    q = q.join(models.PostTag.tag)
    q = q.filter(models.Tag.tag.in_(search_tags))
    q = q.group_by(models.Post)
#     q = q.having(db.func.count(models.Tag.tag) >= len(search_tags))
    q = q.having(db.func.count(models.Tag.tag) >= 1)
    
    results = q.all()
    output = []
    for r in results:
        post, count = r
        post_tags = [p.tag.tag.lower() for p in post.post_tags]
        upvotes = len(post.upvotes)
        post_tags.sort()
        output.append((post, count, upvotes, post_tags))

print('search:')
print('\t{}'.format(search_tags))
print('results:')
for post, count, upvotes, post_tags in output:
    msg = '\tpost: {id}, count: {count}, votes: {votes} <- {tags}'.format(
        id = str(post.id).zfill(2),
        count = count,
        tags = ', '.join(post_tags),
        votes=upvotes
    )
    print(msg)

search:
	['deploy', 'microservice', 'apps', 'in', 'GCP', 'with', 'docker', 'kubernetes', 'deploying']
results:
	post: 01, count: 1, votes: 0 <- 0, containers, docker, healthtech, post
	post: 03, count: 2, votes: 0 <- 2, apps, docker, healthtech, post
	post: 06, count: 1, votes: 0 <- 5, docker, healthtech, post, tutorial
	post: 07, count: 1, votes: 0 <- 6, apps, healthtech, hippa, post
	post: 08, count: 1, votes: 0 <- 7, apps, aws, post, tutorial
	post: 09, count: 1, votes: 0 <- 8, apps, containers, post, tutorial
